In [4]:
import matplotlib.pyplot as plt
import os
import numpy as np
%matplotlib qt

In [5]:
def cut_lfp_dat(p,path_out,n_channels,start,stop,frequency = 20000):
    bytes_size = 2
    start_index = int(start*frequency*n_channels*bytes_size)
    stop_index = int(stop*frequency*n_channels*bytes_size)
    #In order not to read after the file
    if stop_index > os.path.getsize(p): stop_index = os.path.getsize(p)
    fp = np.memmap(p, np.int16, 'r', start_index, shape = (stop_index - start_index)//bytes_size)
    data = np.array(fp).reshape(len(fp)//n_channels, n_channels)
#     plt.plot(data[::16,89])
    return data


In [6]:
save_folder = "Z:/Tmp/"

In [7]:
rat_03_path = 'Z:/Rat03-vHPC/Rat03-20201007/Rat03-20201007.dat'
rat_03 = {
#           'easy_ripples' : [17450,17800],
#           'REM'          : [17100,17160],
#           'REM2'         : [18180,18333],
#           'REM3'         : [18639,18802],
#           'unplugged'    : [19840,19900],
          'plugged'      : [20300,20360]
}
rat_08_path = 'Z:/Rat08/Rat08-20130713/Rat08-20130713.dat'
rat_08 = {
          'hard_ripples' : [17580,17800]
}

In [9]:
# for n,times in rat_03.items():
#     data = cut_lfp_dat(rat_03_path,'',102,times[0],times[1])
#     with open(f'Z:/Tmp/{n}.dat','wb') as f:
#         f.write(data)
#         f.close()
#     np.save(f'Z:/Tmp/{n}.npy',data)

    
for n,times in rat_08.items():
    data = cut_lfp_dat(rat_08_path,'',166,times[0],times[1])
    with open(f'Z:/Tmp/{n}.dat','wb') as f:
        f.write(data)
        f.close()
    np.save(f'Z:/Tmp/{n}.npy',data)
    

In [21]:
data = cut_lfp_dat(rat_03_path,'',102,rat_03['REM'][0],rat_03['REM'][1])

In [19]:
def concat_lfp_dat(session,path_dat):
    
    print('Starting dat concatenation')
    
    to_cat = " + ".join(path_dat)
    p = subprocess.Popen('copy /B ' + to_cat + " " + session+'.dat&',shell = True)
    
    #Check if concatenated file is not corrupted : 
    originalFileSize = []
    for p in path_dat:
        originalFileSize.append(os.path.getsize(p))
    finalSize = np.sum(originalFileSize)
    f = False
    while not f:
        f = os.path.exists(session+'.dat')
        
    pbar = tqdm(total = finalSize,desc = 'Concat : ')
    concatSize = 0
    while concatSize < finalSize:
        time.sleep(10)
        s = os.path.getsize(session+'.dat')
        
        if concatSize == s:
            print('Concatenation in stuck try again')
            p.kill()
            break
        
        pbar.update(s-concatSize)
        concatSize = s
 
    originalFileSize = []
    for p in path_dat:
        originalFileSize.append(os.path.getsize(p))
    finalSize = np.sum(originalFileSize)
    concatSize = os.path.getsize(session+'.dat')
    
    
    if finalSize == concatSize: 
        print('Concat Succefull')
        print('Creating cat event file')
        
        t_rec = []
        for p in path_dat:
            t = os.path.getsize(p)/(fs*nChannels*bytePerSample)
            print(t)
            t_rec.append(t)
        concat_event = np.cumsum(t_rec)*1000 # 1000 is for conversion in ms
        
        with open(session+'.cat.evt','w') as f:
            for c in concat_event:
                f.write(str(c)+ ' cat\n')
        return 1
    else:
        print('Error, please try again. Check freespace on your drives. You need at least',np.sum(originalFileSize)/1e9,'GB')
        return 0